# Setup

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

# Add Message History (Memory)

The `RunnableWithMessageHistory` lets us **add message history to certain types of chains**.

It:
- wraps another Runnable,
- manages the chat message history for it.

Specifically, it can be used for any runnable that:
-  **takes as input one of**:
    - a sequence of `BaseMessage`,
    - a `dict` with a key that takes a sequence of `BaseMessage`,
    - a `dict` with a key that takes:
        - the latest message(s) as a string or sequence of `BaseMessage`, and
        - a separate key that takes historical messages.
- **and returns as output one of**:
    - a string that can be treated as the contents of an `AIMessage`,
    - a sequence of `BaseMessage`,
    - a `dict` with a key that contains a sequence of `BaseMessage`.

Let's take a look at some examples to see how it works.

First, we construct a runnable which, here:
- accepts a `dict` as input,
- returns a message as output.

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

In [4]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You're an assistant who's good at {ability}. Respond in 20 words or fewer."
    ),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])
runnable = prompt | model

> **API Reference**:
> - [**ChatPromptTemplate**](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)
> - [**MessagesPlaceholder**](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)
> - [**ChatOpenAI**](https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)

To manage the message history, we will need:
1. this runnable,
2. a callable that returns an instance of `BaseChatMessageHistory`.

You can check out the [**memory integrations**](https://integrations.langchain.com/memory) page for implementations of chat message histories using Redis and other providers.

Here, we will demonstrate using an **in-memory** `ChatMessageHistory` as well as more persistent storage using `RedisChatMessageHistory`.

# In-Memory

Below, we show a simple example in which the chat history lives in memory, in this case, via a global python `dict`.

We construct a callable `get_session_history` that references this `dict` to return an instance of `ChatMessageHistory`.

The arguments to the callable can be specified by passing a **configuration** to the `RunnableWithMessageHistory` at runtime.

By default, the configuration parameter is expected to be a single string `session_id`.

This can be adjusted via the `history_factory_config` kwarg.

Using the signel-parameter default...

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [6]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [7]:
with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "history",
)

> **API Reference**: WARNING, THESE LINKS HAVE TO BE FIXED
> - [**ChatMessageHistory**](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.ChatMessageHistory.html)
> - [**BaseChatMessageHistory**](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html)
> - [**RunnableWithMessageHistory**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)

Note that we have specified:
- `input_message_key` (the key to be treated as the latest input message),
- `history_message_key` (the key to add historical messages to)

When invoking this new runnable, we specify the corresponding chat history via a **configuration** parameter:

In [8]:
with_message_history.invoke(
    {
        "ability": "math",
        "input": "What does cosine mean?"
    },
    # The following is mandatory, except, of course, the session_id's value
    config = {"configurable": {"session_id": "abc123"}}
)

AIMessage(content='A trigonometric function that gives the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 34, 'total_tokens': 57}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-384752b0-4a9b-4542-bf38-6df6ffd007ac-0', usage_metadata={'input_tokens': 34, 'output_tokens': 23, 'total_tokens': 57})

> **And now it *remembers*...**

In [9]:
with_message_history.invoke(
    {
        "ability": "math",
        "input": "What?"
    },
    config = {"configurable": {"session_id": "abc123"}}
)

AIMessage(content='Cosine is a function in trigonometry that relates the angle of a right triangle to its side lengths.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 67, 'total_tokens': 89}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65223bcc-d937-42b9-bbc2-37b948350800-0', usage_metadata={'input_tokens': 67, 'output_tokens': 22, 'total_tokens': 89})

> **Obviously, with a new `session_id`, it doesn't remember...**

In [10]:
with_message_history.invoke(
    {
        "ability": "math",
        "input": "What?"
    },
    config = {"configurable": {"session_id": "def234"}}
)

AIMessage(content="I can help with math problems. Just ask me anything math-related, and I'll do my best to assist you.", response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 31, 'total_tokens': 55}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d289ea11-70e3-4367-a8fb-ec68da065fde-0', usage_metadata={'input_tokens': 31, 'output_tokens': 24, 'total_tokens': 55})

The configuration parameters by which we track messaeg histories **can be customized by passing in a `list` of `ConfigurableFieldSpec` objects to the `history_factory_config` parameter.

Below, we use two parameters:
- `user_id`,
- `conversation_id`.

In [11]:
from langchain_core.runnables import ConfigurableFieldSpec

In [12]:
store = {}

def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]

In [14]:
with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "history",
    # Here's the config
    history_factory_config = [
        ConfigurableFieldSpec(
            id = "user_id",
            annotation = str,
            name = "User ID",
            description = "Unique identifier for the user.",
            default = "",
            is_shared = True,
        ),
        ConfigurableFieldSpec(
            id = "conversation_id",
            annotation = str,
            name = "Conversation ID",
            description = "Unique identifier for the conversation.",
            default = "",
            is_shared = True,
        ),
    ],
)

> **API Reference**:
> - [**ConfigurableFieldSpec**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.utils.ConfigurableFieldSpec.html)

In [15]:
with_message_history.invoke(
    {
        "ability": "math",
        "input": "Hello"
    },
    config = {"configurable": {
        "user_id": "123",
        "conversation_id": "1"
    }},
)

AIMessage(content='Hello! How can I help you with math today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 30, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c40797cb-bb71-4fd8-a7c0-153c78d62e05-0', usage_metadata={'input_tokens': 30, 'output_tokens': 11, 'total_tokens': 41})